In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain.schema import Document

# from pickle import the text list (all the article from code civil)
import pickle
import os

from langchain.vectorstores import Qdrant
import qdrant_client


In [5]:

# read .key file to get openai api key
with open('openai_api_key.key', 'r') as f:
    openai_api_key = f.read().strip()

embeddings = OpenAIEmbeddings(
        model='text-embedding-ada-002', openai_api_key=openai_api_key)



In [4]:


path_data_preprocess = "/home/loilibre_data/data_preprocess/"

# we list all the pickle file in the folder
list_pickle = os.listdir(path_data_preprocess)

# now we only want to take the pickle file that have "short" in their name
list_pickle = [x for x in list_pickle if "short" in x]

list_pickle = [x for x in list_pickle if "Codepenal" in x]

# now (for testing) we only take the Codecivil pickle file
# list_pickle = [x for x in list_pickle if "Codecivil" in x]

# now we read all the pickel file and put them in a list
text_list = []

for pickle_file in list_pickle:
    
    with open(path_data_preprocess + pickle_file, 'rb') as f:
            
        tmp_list = pickle.load(f)

        # we add the type of code for each article 
        # (we remove the .pickle at the end)
        # 
        tmp_list = [pickle_file[:-7] + " " + x for x in tmp_list]

        text_list += tmp_list


In [5]:
text_list

['Codepenal_short 111-1  Loi 92-683 1992-07-22  -  Conseil Constit. 2022-1023 QPC    ARTICLES_STAMP Les infractions pénales sont classées, suivant leur gravité, en crimes, délits et contraventions.',
 'Codepenal_short 111-2  Loi 92-683 1992-07-22  -  Conseil Constit. 2022-1023 QPC    ARTICLES_STAMP La loi détermine les crimes et délits et fixe les peines applicables à leurs auteurs. Le règlement détermine les contraventions et fixe, dans les limites et selon les distinctions établies par la loi, les peines applicables aux contrevenants.',
 "Codepenal_short 111-3  Loi 92-683 1992-07-22  -  Conseil Constit. 2022-1023 QPC    ARTICLES_STAMP Nul ne peut être puni pour un crime ou pour un délit dont les éléments ne sont pas définis par la loi, ou pour une contravention dont les éléments ne sont pas définis par le règlement. Nul ne peut être puni d'une peine qui n'est pas prévue par la loi, si l'infraction est un crime ou un délit, ou par le règlement, si l'infraction est une contravention. 1

In [6]:


# we create a list of Document object
document_list = [Document(page_content=text, metadata={"source" : text[:60]}) for text in text_list]

In [9]:
# now we add every element of the list in the vectorstore (embedding style)
qdrant = Qdrant.from_documents(
    document_list, embeddings, 
    path="../vectorstore",
    collection_name="my_documents",
)

In [1]:
client = qdrant_client.QdrantClient(
    path="../vectorstore", prefer_grpc=True
)

In [6]:
qdrant = Qdrant(
    client=client, collection_name="my_documents", 
    embedding_function=embeddings.embed_query
)

In [7]:
query = "J'ai tué un homme avec un couteau. Il est mort sur le coup. Il s'attaquait à ma famille. Je ne regrette rien."
found_docs = qdrant.similarity_search_with_score(query, k=4)